# Teva - List
## Wichtig
* PDF-Konvertierung mit Tabula kommt immer schräg raus. Daher wird das PDF in eine HTML-Tabelle konvertiert. Dazu müssen aber händisch einige Schritte vorgenommen werden. Script folgen.
* HCP & HCO manuell setzen!

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import numpy as np
import pandas as pd
import importlib
import pikepdf

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Unlock PDF
Dieser Teil entfernt das Lock des PDFs, damit das PDF in HTML exportiert werden kann

In [2]:
pdf = pikepdf.open('PKK Erfassung.Teva Pharma AG.20180427_ENG_Final.pdf')
pdf.save('unlocked.pdf')

## PDF to HTML
Dieser Schritt muss **manuell** getätigt werden!
* PDF `unlocked.pdf` in Adobe Acrobat (nicht Reader!) öffnen
* Datei -> Exportieren in -> HTML Website
* Speichern als `unlocked.html`
* Bilder löschen

## Import HTML

In [3]:
df_list = pd.read_html("unlocked.html", "")
df = pd.concat(df_list)

## Format Table

In [4]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'date', 'donations_grants', 'empty1', 'empty2', 'sponsorship', 'registration_fees','travel_accommodation', 'empty3', 'fees', 'related_expenses', 'total', 'empty4']

#Shift
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Remove rows which have no values
df_export = df_export.dropna(subset=['name'], how='all')

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export[df_export['country'] != '‐']

#Add Type
df_export = df_export.reset_index(drop=True)
index_hco = df_export[df_export['name'].str.contains("GRSSGO", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')
add_warning(manually=True)

#Remove Title
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Revert Name
df_export = revert_name(df_export, ', ')

#Remove empty
df_export.drop(columns=['date', 'empty1', 'empty2', 'empty3', 'empty4'], inplace=True)

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Add Fields
df_export = add_uci(df_export)
df_export = add_plz(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Export
export_list(df_export, 'teva')

Duplicates found. Please check for duplicates: df_export[df_export.duplicated()]
Total nicht Summe der Werte
saved


In [24]:
df_export[df_export.duplicated()]

,name,location,country,address,plz,donations_grants,uci,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,type,source
162,"Universitätsspital Basel, Onkologie",Basel,CH,Petersgraben 4,NaN,NaN,NaN,1500.0,NaN,NaN,NaN,NaN,1500,hco,teva
176,"Universität Zürich, Klinik für Neurologie",Zürich,CH,Frauenklinikstrasse 26,NaN,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,4000,hco,teva
188,Ospedale Regionale di Lugano,Lugano,CH,Via Tesserete 46,NaN,NaN,NaN,2500.0,NaN,NaN,NaN,NaN,2500,hco,teva


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')